In [1]:
import os, pathlib, datetime as dt
import pandas as pd
from dotenv import load_dotenv
import numpy as np

load_dotenv()
RAW = pathlib.Path(os.getenv('DATA_DIR_RAW', 'data/raw'))
PROC = pathlib.Path(os.getenv('DATA_DIR_PROCESSED', 'data/processed'))
RAW.mkdir(parents=True, exist_ok=True)
PROC.mkdir(parents=True, exist_ok=True)
print('RAW ->', RAW.resolve())
print('PROC ->', PROC.resolve())

RAW -> C:\Users\prabh\Bootcamp_Prabhinder_Singh_Sahni\homework\stage05_data-storage\data\raw
PROC -> C:\Users\prabh\Bootcamp_Prabhinder_Singh_Sahni\homework\stage05_data-storage\data\processed


In [2]:
dates = pd.date_range('2024-01-01', periods=20, freq='D')
df = pd.DataFrame({'date': dates, 'ticker': ['AAPL']*20, 'price': 150 + np.random.randn(20).cumsum()})
df.head()

,date,ticker,price
0,2024-01-01,AAPL,149.048298
1,2024-01-02,AAPL,147.926639
2,2024-01-03,AAPL,148.005283
3,2024-01-04,AAPL,150.253638
4,2024-01-05,AAPL,152.580282


In [3]:
def ts(): return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

# TODO: Save CSV
csv_path = RAW / f"sample_{ts()}.csv"
df.to_csv(csv_path, index=False)
csv_path

# TODO: Save Parquet
pq_path = PROC / f"sample_{ts()}.parquet"
try:
    df.to_parquet(pq_path)
except Exception as e:
    print('Parquet engine not available. Install pyarrow or fastparquet to complete this step.')
    pq_path = None
pq_path

WindowsPath('data/processed/sample_20250821-032956.parquet')

In [4]:
def validate_loaded(original, reloaded):
    checks = {
        'shape_equal': original.shape == reloaded.shape,
        'date_is_datetime': pd.api.types.is_datetime64_any_dtype(reloaded['date']) if 'date' in reloaded.columns else False,
        'price_is_numeric': pd.api.types.is_numeric_dtype(reloaded['price']) if 'price' in reloaded.columns else False,
    }
    return checks

df_csv = pd.read_csv(csv_path, parse_dates=['date'])
validate_loaded(df, df_csv)

{'shape_equal': True, 'date_is_datetime': True, 'price_is_numeric': True}

In [5]:
if pq_path:
    try:
        df_pq = pd.read_parquet(pq_path)
        validate_loaded(df, df_pq)
    except Exception as e:
        print('Parquet read failed:', e)

In [6]:
import typing as t, pathlib

def detect_format(path: t.Union[str, pathlib.Path]):
    s = str(path).lower()
    if s.endswith('.csv'): return 'csv'
    if s.endswith('.parquet') or s.endswith('.pq') or s.endswith('.parq'): return 'parquet'
    raise ValueError('Unsupported format: ' + s)

def write_df(df: pd.DataFrame, path: t.Union[str, pathlib.Path]):
    p = pathlib.Path(path); p.parent.mkdir(parents=True, exist_ok=True)
    fmt = detect_format(p)
    if fmt == 'csv':
        df.to_csv(p, index=False)
    else:
        try:
            df.to_parquet(p)
        except Exception as e:
            raise RuntimeError('Parquet engine not available. Install pyarrow or fastparquet.') from e
    return p

def read_df(path: t.Union[str, pathlib.Path]):
    p = pathlib.Path(path)
    fmt = detect_format(p)
    if fmt == 'csv':
        return pd.read_csv(p, parse_dates=['date']) if 'date' in pd.read_csv(p, nrows=0).columns else pd.read_csv(p)
    else:
        try:
            return pd.read_parquet(p)
        except Exception as e:
            raise RuntimeError('Parquet engine not available. Install pyarrow or fastparquet.') from e

# Demo
p_csv = RAW / f"util_{ts()}.csv"
p_pq  = PROC / f"util_{ts()}.parquet"
write_df(df, p_csv); read_df(p_csv).head()
try:
    write_df(df, p_pq)
    read_df(p_pq).head()
except RuntimeError as e:
    print('Skipping Parquet util demo:', e)

print(df)

         date ticker       price
0  2024-01-01   AAPL  149.048298
1  2024-01-02   AAPL  147.926639
2  2024-01-03   AAPL  148.005283
3  2024-01-04   AAPL  150.253638
4  2024-01-05   AAPL  152.580282
5  2024-01-06   AAPL  152.763248
6  2024-01-07   AAPL  152.666396
7  2024-01-08   AAPL  154.616462
8  2024-01-09   AAPL  155.569459
9  2024-01-10   AAPL  154.969729
10 2024-01-11   AAPL  154.956808
11 2024-01-12   AAPL  155.231122
12 2024-01-13   AAPL  153.119308
13 2024-01-14   AAPL  153.155054
14 2024-01-15   AAPL  152.285569
15 2024-01-16   AAPL  151.737210
16 2024-01-17   AAPL  151.145525
17 2024-01-18   AAPL  151.144604
18 2024-01-19   AAPL  150.031588
19 2024-01-20   AAPL  150.694840
